In [1]:
import re
import unicodedata
from pathlib import Path

In [4]:
# 경로 설정
WHITELIST_PATH = Path("../data/whitelist.txt")   # 한 줄에 한 단어씩
INPUT_PATH = Path("../corpus/total_corpus.txt")           # 원본 텍스트 파일
OUTPUT_PATH = Path("../corpus/total_corpus_remove_particle.txt") # 조사 제거 후 저장할 파일

In [7]:
# 화이트리스트 파일 가져오기

def load_whitelist(path: Path):
    words = set()
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            w = line.strip()
            if not w:
                continue
            # 유니코드 정규화 (NFC로 통일)
            w = unicodedata.normalize("NFC", w)
            words.add(w)
    return words

whitelist = load_whitelist(WHITELIST_PATH)
print("화이트리스트 개수:", len(whitelist))

화이트리스트 개수: 68


In [8]:
# 조사 제거 준비
JOSA_FIRST_CHARS = set("은는이가을를와과에로도만뿐께랑한부터까지조")

def build_patterns(whitelist):
    patterns = []
    for base in whitelist:
        # base 뒤에 1~3글자 한글(josa) + 그 뒤에는 더 이상 한글이 안 나오는 경우만 매치
        # 예: "싸피티비에서", "월말평가는", "입실설문으로"
        pat = re.compile(rf"{re.escape(base)}(?P<josa>[가-힣]{{1,3}})(?![가-힣])")

        def make_repl(b):
            def repl(m):
                josa = m.group("josa")
                # 첫 글자가 흔한 조사 시작이면 → 조사라고 보고 제거
                if josa and josa[0] in JOSA_FIRST_CHARS:
                    return b
                # 아니면 그대로 둔다
                return m.group(0)
            return repl

        patterns.append((pat, make_repl(base)))
    return patterns

patterns = build_patterns(whitelist)
print("패턴 개수:", len(patterns))

def strip_josa_in_text(text: str) -> str:
    """화이트리스트 단어 뒤에 붙은 조사(1~3글자)를 조건부로 잘라낸다."""
    text = unicodedata.normalize("NFC", text)
    for pat, repl in patterns:
        text = pat.sub(repl, text)
    return text

패턴 개수: 68


In [9]:
test = "어제 싸피티비에서 올라온 월말평가는 진짜 재밌었다. 오늘은 입실설문을 잊어버릴 뻔했다. 싸피티비가 알려준 꿀팁 덕분에 살았다."
print("원본:", test)
print("처리:", strip_josa_in_text(test))

원본: 어제 싸피티비에서 올라온 월말평가는 진짜 재밌었다. 오늘은 입실설문을 잊어버릴 뻔했다. 싸피티비가 알려준 꿀팁 덕분에 살았다.
처리: 어제 싸피티비 올라온 월말평가 진짜 재밌었다. 오늘은 입실설문 잊어버릴 뻔했다. 싸피티비 알려준 꿀팁 덕분에 살았다.


In [10]:
# 제거
with INPUT_PATH.open("r", encoding="utf-8") as fin, \
     OUTPUT_PATH.open("w", encoding="utf-8") as fout:
    
    line_count = 0
    for line in fin:
        line = line.rstrip("\n")
        cleaned = strip_josa_in_text(line)
        fout.write(cleaned + "\n")
        line_count += 1

print(f"처리 완료! 총 {line_count}줄 → {OUTPUT_PATH}")

처리 완료! 총 1978줄 → ..\corpus\total_corpus_remove_particle.txt
